## Загрузка данных и подключение библиотек

In [1]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c tabular-playground-series-may-2021
!pip install catboost
!pip install eli5
!pip install optuna
!pip install shap
!pip install scikit-learn-extra
!unzip /content/test.csv.zip
!unzip /content/train.csv.zip
!unzip /content/sample_submission.csv.zip

kaggle.json
  0% 0.00/1.72M [00:00<?, ?B/s]
100% 1.72M/1.72M [00:00<00:00, 119MB/s]
  0% 0.00/851k [00:00<?, ?B/s]
100% 851k/851k [00:00<00:00, 115MB/s]
  0% 0.00/128k [00:00<?, ?B/s]
100% 128k/128k [00:00<00:00, 108MB/s]
     |████████████████████████████████| 67.3MB 45kB/s 
     |████████████████████████████████| 112kB 7.6MB/s 
     |████████████████████████████████| 296kB 10.9MB/s 
     |████████████████████████████████| 174kB 51.9MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
     |████████████████████████████████| 81kB 12.7MB/s 
     |████████████████████████████████| 143kB 57.1MB/s 
     |████████████████████████████████| 112kB 52.6MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=7ae32aa6d0d8757e84e28b56991e537856f7cdb4bea06ad87dc2d779cdff9895
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully bui

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import catboost as cb
import sklearn
import xgboost as xgb
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA, FactorAnalysis as FA
from typing import List, Optional
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split
from typing import List, Tuple
import scipy.stats as ss
from sklearn_extra.cluster import KMedoids
import math
from sklearn.utils.validation import check_is_fitted
import eli5
from sklearn.base import BaseEstimator, TransformerMixin
import time
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import shap
import missingno as msno
from sklearn.inspection import permutation_importance
from eli5.sklearn import PermutationImportance
import optuna
from sklearn.metrics import log_loss
pd.plotting.register_matplotlib_converters()
%matplotlib inline
sns.set(color_codes=True)
pal = sns.color_palette("viridis", 10)
sns.set_palette(pal)

## Используемые функции

In [3]:
def get_input(data_path: str) -> pd.DataFrame:
  """
  Считывание данных и вывод основной информации о наборе данных.

  Parmeters
  ---------
  data_path: str - название файла

  Returns
  -------
  data: pandas.core.frame.DataFrame - загруженный набор данных в pandas.Dataframe
  """
  base_path = "/content"
  data = pd.read_csv(f"{base_path}/{data_path}")
  data.columns = [col.lower() for col in data.columns]
  print(f"{data_path}: shape = {data.shape[0]} rows, {data.shape[1]} cols")
  return data

In [4]:
def plot_feature_importance(importance, names, model_type, figsize=(10,8)):

  #Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=figsize)
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title(model_type + ' FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')
  return list(fi_df['feature_names'].values)

In [5]:
def multi_estimators_predict(estimators: List,
                             x_valid: pd.DataFrame,
                             y_valid = None,
                             metric: callable = None,
                             scalers: List = None):
  preds = []
  evals = []
  if scalers:
    assert len(estimators) == len(scalers)

  for i in range(len(estimators)):
    if scalers:
      pred = estimators[i].predict_proba(scalers[i].transform(x_valid))
    else:
      if type(estimators[0]) == xgb.core.Booster:
        pred = estimators[i].predict(x_valid)
      else:
        pred = estimators[i].predict_proba(x_valid)

    
    preds.append(pred)
    if (y_valid is not None) and (metric is not None):
      eval = metric(y_valid, pred)
      evals.append(eval)
  
  result = np.stack([preds], axis=0)
  result = np.squeeze(result, axis=0)
  result = np.mean(result, axis=0)

  if (y_valid is not None) & (metric is not None):
    for i, eval in enumerate(evals):
      print(f"Model {i} metric: {eval:.7}")
    print(f"Result model metric: {metric(y_valid, result):.7}")

  return result

In [6]:
def catboost_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели catboost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    if not categorical:
        categorical = "auto"

    estimators, folds_scores = [], []
    oof_preds = np.zeros((X.shape[0], 4))
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = cb.CatBoostClassifier(**params)
        model.fit(
            x_train, y_train,
            #eval_set=[(x_valid, y_valid)], 
            verbose=10, 
            #early_stopping_rounds=100,
            #cat_features=categorical
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [7]:
def lightgbm_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели lightgbm.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    if not categorical:
        categorical = "auto"

    estimators, folds_scores = [], []
    oof_preds = np.zeros((X.shape[0], 4))
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = lgb.LGBMClassifier(**params)
        model.fit(
            x_train, y_train,
            #eval_set=[(x_valid, y_valid)],
            #eval_metric="multi_logloss", 
            verbose=10, 
            #early_stopping_rounds=50,
            #categorical_feature=categorical
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

In [8]:
def xgboost_cv_fit(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели xgboost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    encoders: dict
        Список с объектами LabelEncoders.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    estimators, encoders = [], {}
    oof_preds = np.zeros((X.shape[0], 4))

    if categorical:
        for feature in categorical:
            encoder = LabelEncoder()
            X[feature] = encoder.fit_transform(X[feature].astype("str").fillna("NA"))
            encoders[feature] = encoder

    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):

        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(
            x_train, y_train,
            #eval_set=[(x_valid, y_valid)],
            #eval_metric="mlogloss", 
            verbose=10, 
            #early_stopping_rounds=50,
        )

        oof_preds[valid_idx] = model.predict_proba(x_valid)
        score = metrics.log_loss(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        estimators.append(model)

    return estimators, encoders, oof_preds

## Загрузка данных и построение моделей

### lightgbm + optuna

In [ ]:
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.35)
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        'num_class': 4,

        'max_depth': trial.suggest_int('max_depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0, 0.13 ),

    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "multi_logloss")
    gbm = lgb.train(
        param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)

    log_loss = sklearn.metrics.log_loss(valid_y, preds)
    return log_loss

In [11]:
# Лучше подбирать гиперпараметры на всей data через кросс валидацию
import optuna.integration.lightgbm as lgb
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    dtrain = lgb.Dataset(data, label=target)

    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        'num_class': 4,

        'max_depth': trial.suggest_int('max_depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.03, 0.13 ),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "multi_logloss")
    result = lgb.cv(
      params=param,
      train_set=dtrain,
      num_boost_round=10000,
      early_stopping_rounds=100,
      #verbose_eval=10,
      stratified=True,
      seed=42,
      metrics="multi_logloss",
      shuffle=True,
      nfold=5
    )
 
    log_loss = result['multi_logloss-mean'][-1] + result['multi_logloss-stdv'][-1]
    return log_loss

In [ ]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=20), direction="minimize"
    )
study.optimize(objective, n_trials=200)

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
print(study.best_params)

{'lambda_l1': 1.45120566383297e-06, 'lambda_l2': 0.003368552965821498, 'num_leaves': 32, 'feature_fraction': 0.4471131328810426, 'bagging_fraction': 0.5888198372400193, 'bagging_freq': 4, 'min_child_samples': 32}


In [ ]:
study.best_value

1.0833801288748757

In [15]:
data = get_input("train.csv")
data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)
target = data['target']
data = data.drop(columns=['target'])

lgb_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    #'num_boost_round': 100,
    'num_class': 4,
    'lambda_l1': 0.02259438646302076,
    'lambda_l2': 3.3834082296901886e-05,
    'num_leaves': 256,
    'feature_fraction': 0.42939073275678896,
    'bagging_fraction': 0.8760623722003144,
    'bagging_freq': 4,
    'min_child_samples': 71,
    'max_depth': 4,
    'learning_rate': 0.07658957460133804,
}

dtrain = lgb.Dataset(data=data, label=target)

result = lgb.cv(
    params=lgb_params,
    train_set=dtrain,
    num_boost_round=2000,
    early_stopping_rounds=100,
    verbose_eval=10,
    stratified=True,
    seed=42,
    metrics="multi_logloss",
    shuffle=True,
    nfold=5
)

train.csv: shape = 100000 rows, 52 cols
[10]	cv_agg's multi_logloss: 1.11339 + 0.00019763
[20]	cv_agg's multi_logloss: 1.10998 + 0.000341567
[30]	cv_agg's multi_logloss: 1.10748 + 0.000399434
[40]	cv_agg's multi_logloss: 1.1054 + 0.000482044
[50]	cv_agg's multi_logloss: 1.10365 + 0.000591791
[60]	cv_agg's multi_logloss: 1.10218 + 0.000701451
[70]	cv_agg's multi_logloss: 1.10096 + 0.000759168
[80]	cv_agg's multi_logloss: 1.09988 + 0.000727296
[90]	cv_agg's multi_logloss: 1.09888 + 0.000727316
[100]	cv_agg's multi_logloss: 1.09803 + 0.000788178
[110]	cv_agg's multi_logloss: 1.09726 + 0.00084155
[120]	cv_agg's multi_logloss: 1.09667 + 0.000892679
[130]	cv_agg's multi_logloss: 1.09616 + 0.000970538
[140]	cv_agg's multi_logloss: 1.09569 + 0.00102404
[150]	cv_agg's multi_logloss: 1.09526 + 0.00106349
[160]	cv_agg's multi_logloss: 1.09494 + 0.0011131
[170]	cv_agg's multi_logloss: 1.09459 + 0.00114867
[180]	cv_agg's multi_logloss: 1.09433 + 0.00113994
[190]	cv_agg's multi_logloss: 1.09407 + 0.

In [19]:
len(result['multi_logloss-mean'])

334

In [9]:
import lightgbm as lgb
data = get_input("train.csv")
test = get_input("test.csv")

data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)

target = data['target']
data = data.drop(columns=['target'])

#train, valid = train_test_split(
#    data, train_size=0.7, shuffle=True, random_state=1,
#)

#train.reset_index(inplace=True, drop=True)
#valid.reset_index(inplace=True, drop=True)

#y_train = train['target']
#x_train = train.drop(columns=['target'])
#y_valid = valid['target']
#x_valid = valid.drop(columns=['target'])

train.csv: shape = 100000 rows, 52 cols
test.csv: shape = 50000 rows, 51 cols


In [42]:
lgb_params = {
    "boosting_type ": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    'n_estimators': 334,
    'num_class': 4,
    'lambda_l1': 0.02259438646302076,
    'lambda_l2': 3.3834082296901886e-05,
    'num_leaves': 256,
    'feature_fraction': 0.42939073275678896,
    'bagging_fraction': 0.8760623722003144,
    'bagging_freq': 4,
    'min_child_samples': 71,
    'max_depth': 4,
    'learning_rate': 0.07658957460133804,
    'random_state': 42,
}

cv = StratifiedKFold(n_splits=7, random_state=435, shuffle=True)

lgb_estimators, lgb_oof = lightgbm_cv_fit(
    lgb_params, data, target, cv, #categorical=categorical_feature_names
)

Tue May 25 17:26:21 2021, Cross-Validation, 100000 rows, 50 cols
Fold 1, Valid score = 1.09316
Fold 2, Valid score = 1.09138
Fold 3, Valid score = 1.09496
Fold 4, Valid score = 1.09132
Fold 5, Valid score = 1.09438
Fold 6, Valid score = 1.09386
Fold 7, Valid score = 1.09048
Score by each fold: [1.09316, 1.09138, 1.09496, 1.09132, 1.09438, 1.09386, 1.09048]


In [43]:
print(f"Out of fold log loss {metrics.log_loss(target, lgb_oof)}")

Out of fold log loss 1.092792546315436


In [44]:
lgb_estimators[0]

LGBMClassifier(bagging_fraction=0.8760623722003144, bagging_freq=4,
               boosting_type ='gbdt', feature_fraction=0.42939073275678896,
               lambda_l1=0.02259438646302076, lambda_l2=3.3834082296901886e-05,
               learning_rate=0.07658957460133804, max_depth=4,
               metric='multi_logloss', min_child_samples=71, n_estimators=334,
               num_class=4, num_leaves=256, objective='multiclass',
               random_state=42)

### Catboost

In [ ]:
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.35)

    param = {
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "task_type": "GPU",
        
        #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        #"subsample": trial.suggest_float("subsample", 0.1, 1),

        'min_data_in_leaf': 25,
        'depth': trial.suggest_int('depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.13 ),
        'random_strength' :trial.suggest_int('random_strength', 0, 100),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),                       
        #'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), # for bayesian bootstrap only

        #'bootstrap_type': 'Bernoulli',
        #'leaf_estimation_method': 'Newton',

        'grow_policy': "SymmetricTree",
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "multi_logloss")
    gbm = cb.CatBoostClassifier(**param)

    gbm.fit(train_x, train_y, eval_set=[(valid_x, valid_y)], verbose=0, early_stopping_rounds=100)
    preds = gbm.predict_proba(valid_x)


    log_loss = sklearn.metrics.log_loss(valid_y, preds)
    return log_loss

In [ ]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=20), direction="minimize"
    )
study.optimize(objective, n_trials=200)

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
data = get_input("train.csv")
data.drop(columns='id', inplace=True)
mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
data['target'] = data['target'].map(mapper)
target = data['target']
data = data.drop(columns=['target'])

cb_params = {
        "loss_function": "MultiClass",
        "eval_metric": "MultiClass",
        "task_type": "GPU",
        'min_data_in_leaf': 25,
        'depth': 3,
        'learning_rate': 0.09,
        'random_strength' :10,
        "l2_leaf_reg": 100,                       
        'grow_policy': "SymmetricTree",
        #"thread_count": 10,
        "random_seed": 27,
    }
cv_result = cb.cv(
        pool = cb.Pool(data, target),
        params = cb_params,
        plot=True,
        shuffle = True,
        stratified = True,
        seed = 42,
        iterations = 10000,
        early_stopping_rounds = 100,
        fold_count = 5,
        as_pandas = True,
        verbose_eval = 10
        )

In [36]:
cb_params = {
    "n_estimators": 1500,
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "task_type": "GPU",
    'min_data_in_leaf': 25,
    'depth': 3,
    'learning_rate': 0.09,
    'random_strength' :10,
    "l2_leaf_reg": 100,                       
    'grow_policy': "SymmetricTree",
    "random_seed": 27,
}

cv = StratifiedKFold(n_splits=7, random_state=435, shuffle=True)

cb_estimators, cb_oof = catboost_cv_fit(
    cb_params, data, target, cv, #categorical=categorical_feature_names
)

Tue May 25 17:18:12 2021, Cross-Validation, 100000 rows, 50 cols


0:	learn: 1.3358583	total: 10.4ms	remaining: 15.6s
10:	learn: 1.1505779	total: 60.7ms	remaining: 8.22s
20:	learn: 1.1186398	total: 111ms	remaining: 7.81s
30:	learn: 1.1109338	total: 165ms	remaining: 7.8s
40:	learn: 1.1086020	total: 216ms	remaining: 7.7s
50:	learn: 1.1068262	total: 266ms	remaining: 7.57s
60:	learn: 1.1056642	total: 320ms	remaining: 7.55s
70:	learn: 1.1045551	total: 370ms	remaining: 7.44s
80:	learn: 1.1034171	total: 420ms	remaining: 7.36s
90:	learn: 1.1022057	total: 466ms	remaining: 7.21s
100:	learn: 1.1012476	total: 514ms	remaining: 7.11s
110:	learn: 1.1006029	total: 558ms	remaining: 6.99s
120:	learn: 1.0996894	total: 610ms	remaining: 6.95s
130:	learn: 1.0989419	total: 661ms	remaining: 6.91s
140:	learn: 1.0983692	total: 704ms	remaining: 6.79s
150:	learn: 1.0976429	total: 749ms	remaining: 6.69s
160:	learn: 1.0970764	total: 800ms	remaining: 6.65s
170:	learn: 1.0964714	total: 842ms	remaining: 6.55s
180:	learn: 1.0957768	total: 892ms	remaining: 6.5s
190:	learn: 1.0953171	to

0:	learn: 1.3359111	total: 5.04ms	remaining: 7.55s
10:	learn: 1.1505913	total: 46.3ms	remaining: 6.27s
20:	learn: 1.1189418	total: 82.2ms	remaining: 5.79s
30:	learn: 1.1112789	total: 118ms	remaining: 5.6s
40:	learn: 1.1088462	total: 157ms	remaining: 5.58s
50:	learn: 1.1071589	total: 191ms	remaining: 5.43s
60:	learn: 1.1059011	total: 228ms	remaining: 5.37s
70:	learn: 1.1048148	total: 266ms	remaining: 5.36s
80:	learn: 1.1036054	total: 302ms	remaining: 5.29s
90:	learn: 1.1025623	total: 337ms	remaining: 5.21s
100:	learn: 1.1016920	total: 372ms	remaining: 5.15s
110:	learn: 1.1008837	total: 405ms	remaining: 5.07s
120:	learn: 1.1001944	total: 443ms	remaining: 5.05s
130:	learn: 1.0994181	total: 490ms	remaining: 5.12s
140:	learn: 1.0988098	total: 527ms	remaining: 5.08s
150:	learn: 1.0982517	total: 563ms	remaining: 5.03s
160:	learn: 1.0976611	total: 607ms	remaining: 5.04s
170:	learn: 1.0970165	total: 647ms	remaining: 5.02s
180:	learn: 1.0965049	total: 688ms	remaining: 5.01s
190:	learn: 1.0959388

0:	learn: 1.3358884	total: 4.55ms	remaining: 6.82s
10:	learn: 1.1504904	total: 38.1ms	remaining: 5.16s
20:	learn: 1.1185129	total: 72.2ms	remaining: 5.08s
30:	learn: 1.1109224	total: 106ms	remaining: 5.01s
40:	learn: 1.1084759	total: 143ms	remaining: 5.11s
50:	learn: 1.1067072	total: 176ms	remaining: 5s
60:	learn: 1.1053921	total: 212ms	remaining: 5.01s
70:	learn: 1.1043735	total: 248ms	remaining: 4.99s
80:	learn: 1.1032953	total: 281ms	remaining: 4.92s
90:	learn: 1.1022209	total: 314ms	remaining: 4.86s
100:	learn: 1.1012515	total: 347ms	remaining: 4.81s
110:	learn: 1.1004379	total: 378ms	remaining: 4.73s
120:	learn: 1.0996012	total: 418ms	remaining: 4.76s
130:	learn: 1.0988661	total: 451ms	remaining: 4.71s
140:	learn: 1.0983006	total: 484ms	remaining: 4.66s
150:	learn: 1.0976099	total: 519ms	remaining: 4.64s
160:	learn: 1.0970181	total: 553ms	remaining: 4.59s
170:	learn: 1.0964116	total: 596ms	remaining: 4.63s
180:	learn: 1.0959082	total: 635ms	remaining: 4.63s
190:	learn: 1.0951957	t

0:	learn: 1.3359278	total: 4.63ms	remaining: 6.93s
10:	learn: 1.1506382	total: 43.9ms	remaining: 5.94s
20:	learn: 1.1188603	total: 77.5ms	remaining: 5.46s
30:	learn: 1.1111931	total: 112ms	remaining: 5.29s
40:	learn: 1.1087210	total: 147ms	remaining: 5.22s
50:	learn: 1.1068368	total: 180ms	remaining: 5.11s
60:	learn: 1.1056512	total: 216ms	remaining: 5.09s
70:	learn: 1.1045368	total: 246ms	remaining: 4.95s
80:	learn: 1.1033043	total: 282ms	remaining: 4.93s
90:	learn: 1.1021539	total: 315ms	remaining: 4.87s
100:	learn: 1.1012387	total: 346ms	remaining: 4.79s
110:	learn: 1.1004472	total: 379ms	remaining: 4.74s
120:	learn: 1.0995990	total: 430ms	remaining: 4.9s
130:	learn: 1.0987913	total: 463ms	remaining: 4.84s
140:	learn: 1.0982679	total: 497ms	remaining: 4.79s
150:	learn: 1.0976635	total: 530ms	remaining: 4.74s
160:	learn: 1.0971624	total: 563ms	remaining: 4.68s
170:	learn: 1.0965186	total: 596ms	remaining: 4.63s
180:	learn: 1.0959844	total: 634ms	remaining: 4.62s
190:	learn: 1.0954923

0:	learn: 1.3358730	total: 6.72ms	remaining: 10.1s
10:	learn: 1.1505497	total: 43.1ms	remaining: 5.83s
20:	learn: 1.1188194	total: 76.4ms	remaining: 5.38s
30:	learn: 1.1111136	total: 110ms	remaining: 5.2s
40:	learn: 1.1086327	total: 146ms	remaining: 5.21s
50:	learn: 1.1067481	total: 180ms	remaining: 5.12s
60:	learn: 1.1055695	total: 229ms	remaining: 5.41s
70:	learn: 1.1043408	total: 261ms	remaining: 5.26s
80:	learn: 1.1031732	total: 295ms	remaining: 5.18s
90:	learn: 1.1020568	total: 329ms	remaining: 5.09s
100:	learn: 1.1010035	total: 362ms	remaining: 5.01s
110:	learn: 1.1001406	total: 395ms	remaining: 4.94s
120:	learn: 1.0993882	total: 434ms	remaining: 4.94s
130:	learn: 1.0986256	total: 467ms	remaining: 4.88s
140:	learn: 1.0980021	total: 506ms	remaining: 4.87s
150:	learn: 1.0972977	total: 539ms	remaining: 4.81s
160:	learn: 1.0967919	total: 573ms	remaining: 4.76s
170:	learn: 1.0961268	total: 609ms	remaining: 4.74s
180:	learn: 1.0954865	total: 649ms	remaining: 4.73s
190:	learn: 1.0950041

0:	learn: 1.3359277	total: 5.08ms	remaining: 7.61s
10:	learn: 1.1506243	total: 41.1ms	remaining: 5.57s
20:	learn: 1.1188291	total: 82.1ms	remaining: 5.78s
30:	learn: 1.1111026	total: 128ms	remaining: 6.06s
40:	learn: 1.1086137	total: 162ms	remaining: 5.75s
50:	learn: 1.1069414	total: 197ms	remaining: 5.6s
60:	learn: 1.1057115	total: 235ms	remaining: 5.54s
70:	learn: 1.1046995	total: 268ms	remaining: 5.39s
80:	learn: 1.1034882	total: 304ms	remaining: 5.32s
90:	learn: 1.1023585	total: 338ms	remaining: 5.24s
100:	learn: 1.1013438	total: 374ms	remaining: 5.18s
110:	learn: 1.1005511	total: 412ms	remaining: 5.16s
120:	learn: 1.0998050	total: 453ms	remaining: 5.16s
130:	learn: 1.0990560	total: 490ms	remaining: 5.12s
140:	learn: 1.0984971	total: 525ms	remaining: 5.06s
150:	learn: 1.0977865	total: 561ms	remaining: 5.01s
160:	learn: 1.0971936	total: 601ms	remaining: 5s
170:	learn: 1.0966266	total: 637ms	remaining: 4.95s
180:	learn: 1.0960141	total: 678ms	remaining: 4.94s
190:	learn: 1.0953949	to

0:	learn: 1.3358846	total: 4.93ms	remaining: 7.39s
10:	learn: 1.1507243	total: 40.3ms	remaining: 5.45s
20:	learn: 1.1190139	total: 74.5ms	remaining: 5.25s
30:	learn: 1.1114697	total: 108ms	remaining: 5.11s
40:	learn: 1.1089209	total: 148ms	remaining: 5.25s
50:	learn: 1.1070854	total: 181ms	remaining: 5.13s
60:	learn: 1.1058186	total: 216ms	remaining: 5.1s
70:	learn: 1.1047403	total: 248ms	remaining: 4.99s
80:	learn: 1.1036698	total: 281ms	remaining: 4.92s
90:	learn: 1.1025064	total: 315ms	remaining: 4.87s
100:	learn: 1.1015779	total: 348ms	remaining: 4.81s
110:	learn: 1.1008387	total: 380ms	remaining: 4.76s
120:	learn: 1.1000601	total: 419ms	remaining: 4.78s
130:	learn: 1.0991770	total: 456ms	remaining: 4.76s
140:	learn: 1.0985502	total: 489ms	remaining: 4.71s
150:	learn: 1.0978550	total: 532ms	remaining: 4.76s
160:	learn: 1.0973444	total: 565ms	remaining: 4.7s
170:	learn: 1.0967436	total: 604ms	remaining: 4.7s
180:	learn: 1.0962030	total: 643ms	remaining: 4.68s
190:	learn: 1.0956459	t

In [37]:
print(f"Out of fold log loss {metrics.log_loss(target, cb_oof)}")

Out of fold log loss 1.0910067145333615


In [14]:
cb_estimators[0]

### XGBoost

In [17]:
def objective(trial):
    data = get_input("train.csv")
    data.drop(columns='id', inplace=True)
    mapper = {'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3}
    data['target'] = data['target'].map(mapper)
    target = data['target']
    data = data.drop(columns=['target'])

    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.35)
    dtrain = xgb.Dataset(train_x, label=train_y)
    dvalid = xgb.Dataset(valid_x, label=valid_y)

    param = {
        'objective': 'multi:softprob',
        'eval_metric': 'mlogloss',
        "verbosity": -1,
        'num_class': 4,
        'max_depth': trial.suggest_int('max_depth', 3,7),
        'learning_rate': trial.suggest_uniform('learning_rate', 0, 0.13 ),
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "multi_logloss")
    gbm = lgb.train(
        param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    log_loss = sklearn.metrics.log_loss(valid_y, preds)
    return log_loss

In [ ]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=20), direction="minimize"
    )
study.optimize(objective, n_trials=200)

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

In [ ]:
dtrain = xgb.DMatrix(
    data=data, label=target
)

xgb_params = {
    "booster": "gbtree",
    "eta": "0.1",
    "max_depth": 3,
    "random_seed": 42,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': 13,
    'learning_rate': 0.08356451010151393, 
    'gamma': 0.02911685058980812, 
    'max_depth': 3, 
    'min_child_weight': 10.748514454096288, 
    'max_delta_step': 2.4474818433727927, 
    'subsample': 0.6445037550866027, 
    'colsample_bytree': 0.07634753656242108, 
    'lambda': 13.663280761461781, 
    'alpha': 21.521205761694137, 
    'max_leaves': 48,
    'num_class': 4,
    'tree_method': 'gpu_hist',
}

cv_result_xgb = xgb.cv(
    params=xgb_params,
    dtrain=dtrain,
    num_boost_round=2000,
    #categorical_feature=categorical_feature_names,
    early_stopping_rounds=100,
    verbose_eval=10,
    stratified=True,
    seed=42,
    metrics="mlogloss",
    shuffle=True,
    nfold=5,
    #maximaze=True
)

In [43]:
cv_result_xgb.shape[0]

1838

In [31]:
xgb_params = {
    "booster": "gbtree",
    "eta": "0.1",
    "max_depth": 3,
    "random_seed": 42,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'random_state': 13,
    'learning_rate': 0.08356451010151393, 
    'gamma': 0.02911685058980812, 
    'max_depth': 3, 
    'min_child_weight': 10.748514454096288, 
    'max_delta_step': 2.4474818433727927, 
    'subsample': 0.6445037550866027, 
    'colsample_bytree': 0.07634753656242108, 
    'lambda': 13.663280761461781, 
    'alpha': 21.521205761694137, 
    'max_leaves': 48,
    'num_class': 4,
    'n_estimators': 700,
    'tree_method': 'gpu_hist',
}

In [32]:
cv = StratifiedKFold(n_splits=7, random_state=435, shuffle=True)

xgb_estimators, xgb_encoders, xgb_oof = xgboost_cv_fit(
    xgb_params, data, target, cv, #categorical=categorical_feature_names
)

Tue May 25 17:14:51 2021, Cross-Validation, 100000 rows, 50 cols
Fold 1, Valid score = 1.09173
Fold 2, Valid score = 1.09059
Fold 3, Valid score = 1.09354
Fold 4, Valid score = 1.09015
Fold 5, Valid score = 1.09265
Fold 6, Valid score = 1.09191
Fold 7, Valid score = 1.08839


In [33]:
print(f"Out of fold log loss {metrics.log_loss(target, xgb_oof)}")

Out of fold log loss 1.0912785871418675


In [45]:
xgb_estimators[0]

XGBClassifier(alpha=21.521205761694137, colsample_bytree=0.07634753656242108,
              eta='0.1', eval_metric='mlogloss', gamma=0.02911685058980812,
              lambda=13.663280761461781, learning_rate=0.08356451010151393,
              max_delta_step=2.4474818433727927, max_leaves=48,
              min_child_weight=10.748514454096288, n_estimators=700,
              num_class=4, objective='multi:softprob', random_seed=42,
              random_state=13, subsample=0.6445037550866027,
              tree_method='gpu_hist')

### Млдель второго уровня - логистическая регрессия

In [46]:
result_lgbm_df = pd.DataFrame(lgb_oof)
result_xgb_df = pd.DataFrame(xgb_oof)
result_cb_df = pd.DataFrame(cb_oof)
result_lgbm_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_valid = pd.concat([result_lgbm_df, result_xgb_df, result_cb_df], axis=1) 

In [51]:
x_train_logreg, x_valid_logreg, y_train_logreg, y_valid_logreg = train_test_split(
    result_valid, target, train_size=0.7, shuffle=True, random_state=30,
)

params = {'random_state': 0, 
          'max_iter': 300, 
          'multi_class': 'multinomial', 
          'solver': 'lbfgs'}

logreg = LogisticRegression(**params)
logreg.fit(x_train_logreg, y_train_logreg)

y_pred_logreg = logreg.predict_proba(x_valid_logreg)
metrics.log_loss(y_valid_logreg, y_pred_logreg)

1.0856954002832073

## Получение результата

In [52]:
result_xgb_test = multi_estimators_predict(xgb_estimators, test.drop(columns='id'))
result_lgb_test = multi_estimators_predict(lgb_estimators, test.drop(columns='id'))
result_cb_test = multi_estimators_predict(cb_estimators, test.drop(columns='id'))

result_lgb_test_df = pd.DataFrame(result_lgb_test)
result_xgb_test_df = pd.DataFrame(result_xgb_test)
result_cb_test_df = pd.DataFrame(result_cb_test)
result_lgb_test_df.rename(columns={0:'lgb_class_1',	1:'lgb_class_2',	2:'lgb_class_3',	3:'lgb_class_4'}, inplace=True)
result_xgb_test_df.rename(columns={0:'xgb_class_1',	1:'xgb_class_2',	2:'xgb_class_3',	3:'xgb_class_4'}, inplace=True)
result_cb_test_df.rename(columns={0:'cb_class_1',	1:'cb_class_2',	2:'cb_class_3',	3:'cb_class_4'}, inplace=True)
result_for_logreg = pd.concat([result_lgb_test_df, result_xgb_test_df, result_cb_test_df], axis=1) 

In [53]:
y_pred_logreg = logreg.predict_proba(result_for_logreg)

In [54]:
test_pred_df = pd.DataFrame(y_pred_logreg)
test_pred_df.rename(columns={0:'class_1',	1:'class_2',	2:'class_3',	3:'class_4'}, inplace=True)
result = test.drop(columns = test.columns.to_list()[1:])
result = pd.concat([result, test_pred_df], axis=1)
result.to_csv('result.csv', index=False)